<a href="https://colab.research.google.com/github/Sampath04x/PRODIGY_ML_04/blob/main/ML_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d gti-upm/leapgestrecog

Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
100% 2.13G/2.13G [00:25<00:00, 74.3MB/s]
100% 2.13G/2.13G [00:25<00:00, 90.7MB/s]


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/leapgestrecog.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten


In [13]:
import os
import pandas as pd
def gather_image_paths_and_labels(root_dir):
    file_paths = []
    labels = []

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                file_paths.append(os.path.join(root, file))
                label = os.path.basename(os.path.dirname(root))
                labels.append(label)

    return file_paths, labels
file_paths, labels = gather_image_paths_and_labels(data_dir)


df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

df.head()


,file_path,label
0,/content/leapGestRecog/03/03_fist/frame_03_03_...,03
1,/content/leapGestRecog/03/03_fist/frame_03_03_...,03
2,/content/leapGestRecog/03/03_fist/frame_03_03_...,03
3,/content/leapGestRecog/03/03_fist/frame_03_03_...,03
4,/content/leapGestRecog/03/03_fist/frame_03_03_...,03


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 224, 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_generator = datagen.flow_from_dataframe(
    df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 16000 validated image filenames belonging to 10 classes.
Found 4000 validated image filenames belonging to 10 classes.


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = len(train_generator.class_indices)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
500/500 [==============================] - 74s 135ms/step - loss: 0.1599 - accuracy: 0.9473 - val_loss: 16.2382 - val_accuracy: 0.0000e+00
Epoch 2/10
500/500 [==============================] - 64s 127ms/step - loss: 0.0243 - accuracy: 0.9886 - val_loss: 24.3555 - val_accuracy: 0.0000e+00
Epoch 3/10
500/500 [==============================] - 69s 139ms/step - loss: 0.0199 - accuracy: 0.9888 - val_loss: 21.1312 - val_accuracy: 0.0000e+00
Epoch 4/10
500/500 [==============================] - 65s 130ms/step - loss: 0.0179 - accuracy: 0.9895 - val_loss: 26.3894 - val_accuracy: 0.0000e+00
Epoch 5/10
500/500 [==============================] - 66s 132ms/step - loss: 0.0230 - accuracy: 0.9898 - val_loss: 39.3545 - val_accuracy: 0.0000e+00
Epoch 6/10
500/500 [==============================] - 68s 135ms/step - loss: 0.0172 - accuracy: 0.9902 - val_loss: 29.0008 - val_accuracy: 0.0000e+00
Epoch 7/10
500/500 [==============================] - 67s 135ms/step - loss: 0.0160 - accuracy: 0.99

In [17]:
validation_loss, validation_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {validation_acc}")


125/125 [==============================] - 12s 97ms/step - loss: 33.1273 - accuracy: 0.0000e+00
Validation Accuracy: 0.0


In [18]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_hand_gesture(img_path, model):
    img = load_img(img_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])

    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    gesture = class_indices[predicted_class]

    return gesture


img_path = '/content/1604979043427.jpeg'
gesture = predict_hand_gesture(img_path, model)
print(f"Predicted Gesture: {gesture}")


1/1 [==============================] - 0s 345ms/step
Predicted Gesture: 05
